## Class-based DNN with Keras

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras # Main keras package
import tensorflow.keras.layers as layers # basic components to build deep learning Models
from tensorflow.keras.datasets import mnist # package for dataset

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# one-hot
nb_classes = 10
Y_train = keras.utils.to_categorical(Y_train, nb_classes)
Y_test = keras.utils.to_categorical(Y_test, nb_classes)
print(Y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
(10000, 10)


In [ ]:
class MyDNN(keras.Model):
  def __init__(self):
    super(MyDNN, self).__init__()
  
  # build : keras convention for creating weights
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.L1 = layers.Dense(256, activation='relu')
    self.L2 = layers.Dense(256, activation='relu')
    self.L3 = layers.Dense(nb_classes)

  # call : specifies computation.
  def call(self, inputs): # automatically called in __call__ after build().
    L1_outputs = self.L1(inputs)
    L2_outputs = self.L2(L1_outputs)
    outputs = self.L3(L2_outputs)

    return outputs

model = MyDNN()

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# define cost/loss & optimizer
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),
              metrics=['accuracy'])

# train my model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=training_epochs,
          verbose=1)

# Test model and check accuracy
score = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 64us/sample - loss: 0.2427 - acc: 0.9287
Epoch 2/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0901 - acc: 0.9720
Epoch 3/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0575 - acc: 0.9820
Epoch 4/15
60000/60000 [==============================] - 2s 37us/sample - loss: 0.0413 - acc: 0.9866
Epoch 5/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0317 - acc: 0.9900
Epoch 6/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0236 - acc: 0.9920
Epoch 7/15
60000/60000 [==============================] - 2s 38us/sample - loss: 0.0214 - acc: 0.9926
Epoch 8/15
60000/60000 [==============================] - 2s 41us/sample - loss: 0.0173 - acc: 0.9944
Epoch 9/15
60000/60000 [==============================] - 2s 41us/sample - loss:

## class-based CNN with Keras

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.datasets import mnist

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# one-hot
nb_classes = 10
Y_train = keras.utils.to_categorical(Y_train, nb_classes)
Y_test = keras.utils.to_categorical(Y_test, nb_classes)
print(Y_test.shape)

(10000, 10)


In [ ]:
class MyCNN(keras.Model):
  def __init__(self):
    super(MyCNN, self).__init__()
    self.dense_initializer = keras.initializers.glorot_uniform
    self.drop_rate = 0.3
  
  # build : keras convention for creating weights
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.layer1 = keras.Sequential([
      layers.Conv2D(32, kernel_size=3, padding="SAME", activation='relu'),
      layers.MaxPool2D(pool_size=2, strides=2),
      layers.Dropout(rate=self.drop_rate) # automatically controls rate when test
    ])

    self.layer2 = keras.Sequential([
      layers.Conv2D(64, kernel_size=3, padding="SAME", activation='relu'),
      layers.MaxPool2D(pool_size=2, strides=2),
      layers.Dropout(rate=self.drop_rate)
    ])

    self.layer3 = keras.Sequential([
      layers.Conv2D(128, kernel_size=3, padding="SAME", activation='relu'),
      layers.MaxPool2D(pool_size=2, strides=2),
      layers.Dropout(rate=self.drop_rate)
    ])
    
    self.flatten = layers.Flatten() # easy reshape to make tensor compatible for fc layer

    self.fc1 = keras.Sequential([
      layers.Dense(625, kernel_initializer=self.dense_initializer, activation='relu'),
      layers.Dropout(rate=self.drop_rate)
    ])

    self.fc2 = layers.Dense(10, kernel_initializer=self.dense_initializer)
    
  # call : specifies computation.
  def call(self, inputs): # automatically called in __call__ after build().
    x = self.layer1(inputs)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.flatten(x)
    x = self.fc1(x)
    outputs = self.fc2(x)

    return outputs

In [ ]:
model = MyCNN()

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# define cost/loss & optimizer
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate),
              metrics=['accuracy'])

# train my model
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=training_epochs,
          verbose=1)

# Test model and check accuracy
score = model.evaluate(X_test, Y_test, batch_size=batch_size)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples
Epoch 1/15
60000/60000 [==============================] - 12s 192us/sample - loss: 0.2588 - acc: 0.9160
Epoch 2/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0771 - acc: 0.9758
Epoch 3/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0572 - acc: 0.9823
Epoch 4/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0496 - acc: 0.9847
Epoch 5/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0436 - acc: 0.9858
Epoch 6/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0384 - acc: 0.9879
Epoch 7/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0352 - acc: 0.9888
Epoch 8/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0324 - acc: 0.9898
Epoch 9/15
60000/60000 [==============================] - 7s 119us/sample - loss: 0.0311 - acc: 0.9899
Epoch 10/15
60000/60000 [========================